In [1]:
import sys
sys.path.append('/home/elmanyer/Documents/BSC/MasterThesis/Code/EQUILI_PY/')

import numpy as np
import matplotlib.pyplot as plt
from src.ElementObject import *
from scipy import optimize
from src.ShapeFunctions import *


In [2]:
ElType = 1
ElOrder = 1
n = 3
dim = 2

# DEFINE RANDOM ELEMENT
Xe = np.array([[-1, -1],[2,0],[0,3]])
#Xe = np.array([[-2, -0.5],[4,1],[0,3]])
Xe = np.array([[0,3],[-2,-3],[2,-2]])
Te = np.array([0,1,2])

Elem = Element(0,ElType,ElOrder,Xe,Te,None,None)

QuadratureOrder = 3
#### QUADRATURE TO INTEGRATE SURFACES (2D)
Xig2DFEM, Elem.Wg2D, Elem.Ng2D = GaussQuadrature(ElType,QuadratureOrder)
# EVALUATE REFERENCE SHAPE FUNCTIONS 
N, dNdxi, dNdeta = EvaluateReferenceShapeFunctions(Xig2DFEM, ElType, ElOrder, n)

# COMPUTE INTERFACE LINEAR APPROXIMATION
def interface(r):
    z = -r+1
    return z

def zedge(r,Xe,edge):
    # FUNCTION DESCRIBING THE RESTRICCION ASSOCIATED TO THE ELEMENT EDGE
    z = ((Xe[edge,1]-Xe[0,1])*r+Xe[0,1]*Xe[edge,0]-Xe[edge,1]*Xe[0,0])/(Xe[edge,0]-Xe[0,0])
    return z

def fun(r,Xe,edge):
    f = interface(r) - zedge(r,Xe,edge)
    return f

def InverseMapping(X, Xe):
    # DEFINE THE NONLINEAR SYSTEM 
    def fun(Xg, X, Xe):
        f = np.array([-X[0],-X[1]])
        for i in range(n):
            Nig, foo, foo = ShapeFunctionsReference(Xg,1, 1, i+1)
            f[0] += Nig*Xe[i,0]
            f[1] += Nig*Xe[i,1]
        return f
    # SOLVE USING NONLINEAR SOLVER
    XG0 = np.array([1/4, 1/4])  # INITIAL GUESS FOR ROOT SOLVER
    sol = optimize.root(fun, XG0, args=(X,Xe))
    #print(sol)
    Xg = sol.x
    return Xg

# COMPUTE PHYSICAL INTERFACE 
Elem.Xeint = np.zeros([2,2])
for i, edge in enumerate([1,2]):
    r0 = 0.5
    sol = optimize.root(fun, r0 , args=(Xe,edge))
    Elem.Xeint[i,:] = [sol.x, interface(sol.x)]
    
# DO TESSELLATION ON PHYSICAL ELEMENT AND GENERATE SUBELEMENTS
Elem.Xemod, Elem.Temod = Element.Tessellation(Mode=0,Xe=Elem.Xe,Xeint=Elem.Xeint)
Elem.Nsub = len(Elem.Temod[:,0])

Elem.SubElements = [Element(index = subelem, ElType = Elem.ElType, ElOrder = Elem.ElOrder,
                                    Xe = Elem.Xemod[Elem.Temod[subelem,:]],
                                    Te = Elem.Te,
                                    LSe = None,
                                    PHIe = None) for subelem in range(Elem.Nsub)]

# 1. MAPP PHYSICAL INTERFACE TO REFERENCE ELEMENT
XiintREF = np.zeros(np.shape(Elem.Xeint))
for i in range(len(Elem.Xeint[:,0])):
    XiintREF[i,:] = Elem.InverseMapping(Elem.Xeint[i,:])
      
# 2. DO TESSELLATION ON REFERENCE ELEMENT
XiREF = np.array([[1,0], [0,1], [0,0]])
XimodREF, TemodREF = Element.Tessellation(Mode=0,Xe=XiREF,Xeint=XiintREF)

# 3. MAP 2D REFERENCE GAUSS INTEGRATION NODES ON THE REFERENCE SUBELEMENTS AND EVALUATE REFERENCE SHAPE FUNCTIONS
for i, subelem in enumerate(Elem.SubElements):
    subelem.Xig2D = N @ XimodREF[TemodREF[i,:]]
    # EVALUATE REFERENCE SHAPE FUNCTIONS ON MODIFIED REFERENCE QUADRATURE
    subelem.N, subelem.dNdxi, subelem.dNdeta = EvaluateReferenceShapeFunctions(subelem.Xig2D, ElType, ElOrder, n)
    # MAPP MODIFIED REFERENCE QUADRATURE ON PHYSICAL ELEMENT
    subelem.Xg2D = subelem.N @ Elem.Xe
    #subelem.Xg2D = N @ subelem.Xe

/tmp/ipykernel_400779/1950177536.py:55: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  Elem.Xeint[i,:] = [sol.x, interface(sol.x)]


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,10))

###### FIRST ROW FIGURES: REFERENCE SPACE
# PLOT REFERENCE INTERFACE LINEAR APPROXIMATION
for i in range(n):
    axs[0,0].plot([XiREF[int(Te[i])-1,0], XiREF[int(Te[int((i+1)%n)])-1,0]], 
            [XiREF[int(Te[i])-1,1], XiREF[int(Te[int((i+1)%n)])-1,1]], color='black', linewidth=3)
axs[0,0].plot(XiintREF[:,0], XiintREF[:,1], '.', color='red',markersize=10)
axs[0,0].plot(XiintREF[:,0], XiintREF[:,1],color='red', linewidth=3)
d = 0.02
for i in range(len(XimodREF[:,0])):
    axs[0,0].text(XimodREF[i,0]+d,XimodREF[i,1]+d,str(i),fontsize=15)

# PLOT TESSELLATION ON PHYSICAL ELEMENT
colorlist = ['blue', 'red', 'green']
markerlist = ['^', '<', '>']
for i in range(Elem.Nsub):
    for j in range(n):   # plot edges
        axs[0,1].plot([XimodREF[TemodREF[i,j],0], XimodREF[TemodREF[i,int((j+1)%n)],0]], [XimodREF[TemodREF[i,j],1], XimodREF[TemodREF[i,int((j+1)%n)],1]],
                 linestyle = '-', color = colorlist[i], linewidth=2 , marker=markerlist[i])
# PLOT NODE NUMERATION
d = 0.02
for i in range(len(XimodREF[:,0])):
    axs[0,1].text(XimodREF[i,0]+d,XimodREF[i,1]+d,str(i),fontsize=15)
# PLOT MODIFIED REFERENCE GAUSS INTEGRATION NODES
for i, subelem in enumerate(Elem.SubElements):
    axs[0,1].plot(subelem.Xig2D[:,0],subelem.Xig2D[:,1],'x',color=colorlist[i], markersize = 8)
    
###### SECOND ROW FIGURES: PHYSICAL SPACE
# PLOT PHYSICAL INTERFACE LINEAR APPROXIMATION
for i in range(n):
    axs[1,0].plot([Xe[int(Te[i])-1,0], Xe[int(Te[int((i+1)%n)])-1,0]], 
            [Xe[int(Te[i])-1,1], Xe[int(Te[int((i+1)%n)])-1,1]], color='black', linewidth=3)

axs[1,0].plot(Elem.Xeint[:,0], Elem.Xeint[:,1], '.', color='red',markersize=10)
axs[1,0].plot(Elem.Xeint[:,0], Elem.Xeint[:,1],color='red', linewidth=3)
# PLOT NODE NUMERATION
d = 0.07
for i in range(len(Elem.Xemod[:,0])):
    axs[1,0].text(Elem.Xemod[i,0]+d,Elem.Xemod[i,1]+d,str(i),fontsize=15)

# PLOT TESSELLATION ON PHYSICAL ELEMENT
for i, subelem in enumerate(Elem.SubElements):
    for j in range(n):   # plot edges
        axs[1,1].plot([subelem.Xe[j,0], subelem.Xe[int((j+1)%n),0]], [subelem.Xe[j,1], subelem.Xe[int((j+1)%n),1]],
                 linestyle = '-', color = colorlist[i], linewidth=2 , marker=markerlist[i])
# PLOT MODIFIED PHYSICAL GAUSS INTEGRATION NODES
for i, subelem in enumerate(Elem.SubElements):
    axs[1,1].plot(subelem.Xg2D[:,0],subelem.Xg2D[:,1],'x',color=colorlist[i], markersize = 8)


In [1]:
import sys
sys.path.append('/home/elmanyer/Documents/BSC/MasterThesis/Code/EQUILI_PY/')

from src.GradShafranovCutFEM import *

### LOCATION OF FOLDERS
folder_MESHES = '/home/elmanyer/Documents/BSC/MasterThesis/Code/EQUILI_PY/MESHES/'   # MESHES
folder_CASES = '/home/elmanyer/Documents/BSC/MasterThesis/Code/EQUILI_PY/CASES/'     # CASES

### SELECT MESH FOLDER...
#MESH = 'TS-UNSTRUCTURED_ULTRACOARSE'
#MESH = 'TS-UNSTRUCTURED_COARSE'
#MESH = 'TS-UNSTRUCTURED_COARSE-FINE'
#MESH = 'TS-UNSTRUCTURED_FINE'
MESH = 'TS-UNSTRUCTURED_FINE_EXTENDED'
#MESH = 'TS-UNSTRUCTURED_ULTRAFINE'

# ... WITH MESH ELEMENTS:
ElementType = 1     # Element type -> 1: TRIANGLE  ; 2: QUADRILATERAL
ElementOrder = 1    # Element order -> 1: LINEAR  ; 2: QUADRATIC ; 3: CUBIC ...

### SELECT SOLUTION CASE FILE:
#CASE = 'TS-FREE-F4E-LINEAR'          # FREE PLASMA BOUNDARY WITH F4E PLASMA REGION GEOMETRY AND LINEAR PLASMA CURRENT
#CASE = 'TS-FREE-F4E-NONLINEAR'       # FREE PLASMA BOUNDARY WITH F4E PLASMA REGION GEOMETRY AND NONLINEAR PLASMA CURRENT MODEL
CASE = 'TS-FREE-F4E-PROFILES'        # FREE PLASMA BOUNDARY WITH F4E PLASMA REGION GEOMETRY AND PLASMA CURRENT MODEL BASED ON PROFILES FOR PRESSURE AND TOROIDAL FIELD
#CASE = 'TS-FREE-F4E-LINEAR-1W'       # FREE PLASMA BOUNDARY WITH F4E PLASMA REGION GEOMETRY, LINEAR PLASMA CURRENT AND FIRST WALL VACUUM VESSEL GEOMETRY  
#CASE = 'TS-FREE-F4E-PROFILES-1W'     # FREE PLASMA BOUNDARY WITH F4E PLASMA REGION GEOMETRY, PLASMA CURRENT MODEL BASED ON PROFILES FOR PRESSURE AND TOROIDAL FIELD AND FIRST WALL VACUUM VESSEL GEOMETRY  

##############################################################

# DECLARE OBJECT PROBLEM 
Problem = GradShafranovCutFEM(folder_MESHES+MESH,folder_CASES+CASE,ElementType,ElementOrder)
Problem.EQUILI()

READ INPUT FILES...
     -> READ MESH DATA FILES...Done!
     -> READ EQUILI DATA FILE...Done!
Done!
INITIALIZATION...
     -> INITIALISE LEVEL-SET...Done!
     -> INITIALISE ELEMENTS...Done!
     -> CLASSIFY ELEMENTS...Done!
     -> APPROXIMATE VACUUM VESSEL FIRST WALL...Done!
     -> APPROXIMATE PLASMA/VACUUM INTERFACE...Done!
     -> COMPUTE NUMERICAL INTEGRATION QUADRATURES...Done!
     -> INITIALISE UNKNOWN VECTORS AND COMPUTE INITIAL GUESS...
         -> COMPUTE INITIAL GUESS FOR PHI_NORM...Done!
         -> COMPUTE INITIAL VACUUM VESSEL FIRST WALL VALUES PHI_B...

IndexError: index 2 is out of bounds for axis 1 with size 2

In [3]:
self = Problem
        
plt.figure(figsize=(9,11))
plt.ylim(1.9,2.5)
plt.xlim(6,6.8)

# PLOT NODES
plt.plot(self.X[:,0],self.X[:,1],'.',color='black')
Tmesh = self.T +1
# PLOT PLASMA REGION ELEMENTS
for elem in self.PlasmaElems:
    ELEMENT = self.Elements[elem]
    # PLOT ELEMENT EDGES
    for i in range(self.n):
        plt.plot([ELEMENT.Xe[i,0], ELEMENT.Xe[(i+1)%ELEMENT.n,0]], [ELEMENT.Xe[i,1], ELEMENT.Xe[(i+1)%ELEMENT.n,1]], color='red', linewidth=1)
    # PLOT QUADRATURE INTEGRATION POINTS
    plt.scatter(ELEMENT.Xg2D[:,0],ELEMENT.Xg2D[:,1],marker='x',c='red')
# PLOT VACCUM ELEMENTS
for elem in self.VacuumElems:
    ELEMENT = self.Elements[elem]
    # PLOT ELEMENT EDGES
    for i in range(self.n):
        plt.plot([ELEMENT.Xe[i,0], ELEMENT.Xe[(i+1)%ELEMENT.n,0]], [ELEMENT.Xe[i,1], ELEMENT.Xe[(i+1)%ELEMENT.n,1]], color='gray', linewidth=1)
    # PLOT QUADRATURE INTEGRATION POINTS
    plt.scatter(ELEMENT.Xg2D[:,0],ELEMENT.Xg2D[:,1],marker='x',c='gray')
# PLOT EXTERIOR ELEMENTS IF EXISTING
for elem in self.ExteriorElems:
    ELEMENT = self.Elements[elem]
    # PLOT ELEMENT EDGES
    for i in range(self.n):
        plt.plot([ELEMENT.Xe[i,0], ELEMENT.Xe[(i+1)%ELEMENT.n,0]], [ELEMENT.Xe[i,1], ELEMENT.Xe[(i+1)%ELEMENT.n,1]], color='black', linewidth=1)
    # PLOT QUADRATURE INTEGRATION POINTS
    plt.scatter(ELEMENT.Xg2D[:,0],ELEMENT.Xg2D[:,1],marker='x',c='black')
    
# PLOT PLASMA/VACUUM INTERFACE ELEMENTS
for elem in self.PlasmaBoundElems:
    ELEMENT = self.Elements[elem]
    # PLOT ELEMENT EDGES
    for i in range(self.n):
        plt.plot([ELEMENT.Xe[i,0], ELEMENT.Xe[(i+1)%ELEMENT.n,0]], [ELEMENT.Xe[i,1], ELEMENT.Xe[(i+1)%ELEMENT.n,1]], color='gold', linewidth=1)
    # PLOT SUBELEMENT EDGES AND INTEGRATION POINTS
    for SUBELEM in ELEMENT.SubElements:
        # PLOT SUBELEMENT EDGES
        for i in range(self.n):
            plt.plot([SUBELEM.Xe[i,0], SUBELEM.Xe[(i+1)%SUBELEM.n,0]], [SUBELEM.Xe[i,1], SUBELEM.Xe[(i+1)%SUBELEM.n,1]], color='gold', linewidth=1)
        # PLOT QUADRATURE INTEGRATION POINTS
        plt.scatter(SUBELEM.Xg2D[:,0],SUBELEM.Xg2D[:,1],marker='x',c='gold')
    # PLOT INTERFACE LINEAR APPROXIMATION AND INTEGRATION POINTS
    for edge in range(ELEMENT.Neint):
        # PLOT INTERFACE LINEAR APPROXIMATION
        plt.plot(ELEMENT.Xeint[edge,:,0], ELEMENT.Xeint[edge,:,1], color='green', linewidth=1)
        # PLOT INTERFACE QUADRATURE
        plt.scatter(ELEMENT.Xgint[edge,:,0],ELEMENT.Xgint[edge,:,1],marker='o',c='green')
        
# PLOT VACUUM VESSEL FIRST WALL ELEMENTS
for elem in self.VacVessWallElems:
    ELEMENT = self.Elements[elem]
    # PLOT ELEMENT EDGES
    for i in range(self.n):
        plt.plot([ELEMENT.Xe[i,0], ELEMENT.Xe[(i+1)%ELEMENT.n,0]], [ELEMENT.Xe[i,1], ELEMENT.Xe[(i+1)%ELEMENT.n,1]], color='darkturquoise', linewidth=1)
    # PLOT QUADRATURE INTEGRATION POINTS
    if self.VACUUM_VESSEL == "COMPUTATIONAL_DOMAIN":
        plt.scatter(ELEMENT.Xg2D[:,0],ELEMENT.Xg2D[:,1],marker='x',c='darkturquoise')
    else:
        for SUBELEM in ELEMENT.SubElements:
            # PLOT SUBELEMENT EDGES
            for i in range(self.n):
                plt.plot([SUBELEM.Xe[i,0], SUBELEM.Xe[(i+1)%SUBELEM.n,0]], [SUBELEM.Xe[i,1], SUBELEM.Xe[(i+1)%SUBELEM.n,1]], color='darkturquoise', linewidth=1)
            # PLOT QUADRATURE INTEGRATION POINTS
            plt.scatter(SUBELEM.Xg2D[:,0],SUBELEM.Xg2D[:,1],marker='x',c='darkturquoise')
    # PLOT INTERFACE LINEAR APPROXIMATION AND INTEGRATION POINTS
    for edge in range(ELEMENT.Neint):
        # PLOT INTERFACE LINEAR APPROXIMATION
        plt.plot(ELEMENT.Xeint[edge,:,0], ELEMENT.Xeint[edge,:,1], color='orange', linewidth=1)
        # PLOT INTERFACE QUADRATURE
        plt.scatter(ELEMENT.Xgint[edge,:,0],ELEMENT.Xgint[edge,:,1],marker='o',c='orange')

plt.show()
